# Import

These requirements are necessary if you launch this notebook from SageMaker instances

In [1]:
"""!pip install mlflow
!pip install pytorch-lightning
!pip install transformers
!pip install tqdm
!pip install sagemaker

!pip install s3fs
!pip install smdebug"""

'!pip install mlflow\n!pip install pytorch-lightning\n!pip install transformers\n!pip install tqdm\n!pip install sagemaker\n\n!pip install s3fs\n!pip install smdebug'

In [2]:
import sys
sys.path.append('../../../')

import os
import sys
import logging
import argparse
from pathlib import Path
from typing import Any, Dict, Optional

In [3]:
from tqdm.auto import tqdm

import torchmetrics
from torchmetrics.functional import accuracy, f1, auroc

import sagemaker
from sagemaker import get_execution_role
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.core.decorators import auto_move_data
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import MLFlowLogger


import matplotlib.pyplot as plt
from pylab import rcParams
from matplotlib import rc

import pandas as pd

Local constants, regarding the data, MLFlow server, paths, etc..: use them

In [4]:
from deep.constants import *
from deep.utils import *

In [5]:
%load_ext autoreload
%autoreload 2

# Data

You can use the data you want. We advise the `pandas` format.

In [6]:
DATA_PATH = os.path.join(
    '..', '..', '..', "data", "frameworks_data", 'data_v0.7','generated_dataset'
)

"""
original_df = pd.read_csv(os.path.join(DATA_PATH, 'full_dataset.csv'))
augmented_data = pd.read_csv(os.path.join(DATA_PATH, 'generated_text.csv'))"""


tot_df = pd.read_csv(os.path.join(DATA_PATH, 'train_val_all.csv'), index_col=0)
test_df = pd.read_csv(os.path.join(DATA_PATH, 'test_v0.7.csv'))

/home/selim/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [7]:
test_df.head()

,entry_id,excerpt,analysis_framework_id,lead_id,project_id,verified,sectors,subpillars_2d,subpillars_1d,geo_location,specific_needs_groups,severity,info_date,demographic_groups,reliability,affected_groups,source_type,url,website
0,489433,Primary and secondary net enrollment rates are...,1306,67488.0,2225.0,False,['Education'],['Humanitarian Conditions->Living Standards'],['Context->Socio Cultural'],['République démocratique du Congo'],[],['Major'],UNKNOWN,"['Children/Youth Male (5 to 17 years old)', 'C...",['Usually reliable'],['Affected'],website,https://blogs.worldbank.org/education/free-pri...,blogs.worldbank.org
1,489435,"After past, partially implemented attempts, th...",1306,67488.0,2225.0,False,['Education'],['Capacities & Response->International Respons...,['Context->Socio Cultural'],['République démocratique du Congo'],[],[],UNKNOWN,"['Children/Youth Male (5 to 17 years old)', 'C...",['Usually reliable'],['Affected'],website,https://blogs.worldbank.org/education/free-pri...,blogs.worldbank.org
2,194719,"[10th November, NW Syria] Now with the lockdow...",1306,43771.0,2028.0,False,[],[],['Covid-19->Restriction Measures'],['Syrian Arab Republic'],[],[],"['10-11-2020', None]",[],['Usually reliable'],[],website,https://reliefweb.int/report/syrian-arab-repub...,www.reliefweb.int
3,186152,El Salvador hace frente a una combinación de c...,829,41125.0,1898.0,False,"['Nutrition', 'WASH', 'Protection', 'Food Secu...",['Humanitarian Conditions->Living Standards'],[],['El Salvador'],[],['Of Concern'],"['14-10-2020', None]",[],['Usually reliable'],"['All', 'Affected']",website,https://www.acaps.org/sites/acaps/files/key-do...,https://www.acaps.org
4,186986,Las personas que habitan en cantones donde no ...,829,42489.0,1184.0,False,['Cross'],['NOT_MAPPED'],[],"['El Oro', 'Huaquillas']",[],['Of Concern'],"['01-08-2020', '31-08-2020']",[],['Usually reliable'],"['Displaced', 'Refugees', 'Migrants', 'Affected']",website,https://reliefweb.int/sites/reliefweb.int/file...,reliefweb.int


In [8]:
"""augmented_data = pd.merge(
    right=original_df.drop(columns=['excerpt']),
    left=augmented_data[['entry_id', 'excerpt']],
    on='entry_id',
    how='right'
)"""

"augmented_data = pd.merge(\n    right=original_df.drop(columns=['excerpt']),\n    left=augmented_data[['entry_id', 'excerpt']],\n    on='entry_id',\n    how='right'\n)"

In [9]:
"""tot_df = pd.concat([original_df, augmented_data])"""

'tot_df = pd.concat([original_df, augmented_data])'

In [10]:
"""from ast import literal_eval"""

'from ast import literal_eval'

In [11]:
"""tot_df['pillars_1d'] = tot_df.subpillars_1d.apply(
    lambda item: [x.split('->')[0] for x in literal_eval(item)]
)
tot_df['pillars_2d'] = tot_df.subpillars_2d.apply(
    lambda item: [x.split('->')[0] for x in literal_eval(item)]
)"""

"tot_df['pillars_1d'] = tot_df.subpillars_1d.apply(\n    lambda item: [x.split('->')[0] for x in literal_eval(item)]\n)\ntot_df['pillars_2d'] = tot_df.subpillars_2d.apply(\n    lambda item: [x.split('->')[0] for x in literal_eval(item)]\n)"

In [12]:
tot_df = tot_df[
    ['entry_id', 'excerpt', 'lead_id',
     'sectors',
     'severity',
     #'demographic_groups',
     'subpillars_1d', 
     'specific_needs_groups',
     'subpillars_2d', 
     #'affected_groups',
     'pillars_2d',
     'pillars_1d'
     
    ]
]

## Sagemaker Prep

### Session

Configure SageMaker

In [13]:
sess = sagemaker.Session(default_bucket=DEV_BUCKET.name)
role = SAGEMAKER_ROLE
role_arn = SAGEMAKER_ROLE_ARN
tracking_uri = MLFLOW_SERVER

### Bucket upload

You need to upload data to an S3 bucket. 




In [14]:
MLFLOW_SERVER

'http://mlflow-deep-387470f3-1883319727.us-east-1.elb.amazonaws.com/'

In [15]:
sample = False  # To make the computations faster, sample = True.

if sample:
    tot_df = tot_df.sample(n=50_000)
    
job_name = f"pytorch-{formatted_time()}-all-models"  # change it as you prefer
input_path = DEV_BUCKET / 'training' / 'input_data' / job_name  # Do not change this

train_path = str(input_path / 'train.pickle')
val_path = str(input_path / 'val.pickle')


tot_df.to_pickle(train_path, protocol=4)  # protocol 4 is necessary, since SageMaker uses python 3.6
test_df.to_pickle(val_path, protocol=4)

### Estimator Definition

In [16]:
# GPU instances

instances = [
    'ml.p2.xlarge',
    'ml.p3.2xlarge'
]

The hyperparameters are passed as command line arguments to the training script. 

You can add/change them as you like. It's important to keep the `tracking_uri` and the `experiment_name` which are used by MLFlow.

The class `PyTorch` is part of the `SageMaker` python API. The parameters are important and you should probably not change most of them. The ones you may want to change are:

- `instance_type`, specify the instance you want
- `source_dir`, specify your script directory. Try to use global variable as much as possible

In [17]:
from sagemaker.pytorch import PyTorch

proportions_negative_examples_test = {
    'sectors': 0.16,
    'demographic_groups': 0.72,
    'subpillars_1d': 0.57,
    'pillars_1d': 0.57,
    'pillars_2d': 0.28,
    'specific_needs_groups': 0.86,
    'subpillars_2d': 0.28,
    'affected_groups': 0.30
}

proportions_negative_examples_train = {
    'sectors':0.02,
    'subpillars_2d':0.05,
    'pillars_2d':0.05,
    'subpillars_1d': 0.57,
    'pillars_1d':0.57,
    'demographic_groups': 0.72,
    'specific_needs_groups': 0.86,
    'affected_groups': 0.15
}
instance_type='ml.p3.2xlarge'

hyperparameters={
    'tracking_uri': MLFLOW_SERVER,
    'experiment_name': "pl-trials",
    'max_len': 256,
    'epochs': 3,
    'model_name': 'microsoft/xtremedistil-l6-h256-uncased',
    'tokenizer_name': 'microsoft/xtremedistil-l6-h256-uncased',
    'dropout_rate': 0.3,
    'pred_threshold':0.4,
    'output_length': 256,
    'learning_rate': 5e-5,
    #'training_names':'sectors,subpillars_2d,pillars_2d,pillars_1d,subpillars_1d,specific_needs_groups,affected_groups,demographic_groups,severity',
    'training_names':'sectors,subpillars_2d,pillars_2d,pillars_1d,subpillars_1d,specific_needs_groups,severity',
    #'training_names':'subpillars_1d,specific_needs_groups,demographic_groups',
    #'training_names':'sectors,subpillars_2d,subpillars_1d,severity,specific_needs_groups,affected_groups,demographic_groups',
    #'train_with_all_positive_examples':True,
    "model_mode":"train",
    "proportions_negative_examples_test": str(proportions_negative_examples_test),
    "proportions_negative_examples_train": str(proportions_negative_examples_train),
    "instance_type": instance_type,
    'beta_f1': 0.8
    #"numbers_augmentation":"with"
}

estimator = PyTorch(
    entry_point='train_mlflow.py',
    source_dir=str('../../../scripts/training/selim/multiclass-lightning'),
    output_path=str(DEV_BUCKET/'models/'),
    code_location=str(input_path),
    instance_type=instance_type,
    instance_count=1,
    role=role,
    framework_version="1.8",
    py_version="py36",
    hyperparameters = hyperparameters,
    job_name=job_name,
    #distribution={"smdistributed": {"dataparallel": {"enabled": False}}}
#     train_instance_count=2,
#     train_instance_type="ml.c4.xlarge",
)

In [18]:
fit_arguments = {
    'train': str(input_path),
    'test': str(input_path)
}

In [19]:
# Fit the estimator

estimator.fit(fit_arguments, job_name=job_name)

2021-10-27 09:31:37 Starting - Starting the training job...
2021-10-27 09:32:00 Starting - Launching requested ML instancesProfilerReport-1635327090: InProgress
......
2021-10-27 09:33:12 Starting - Preparing the instances for training............
2021-10-27 09:35:11 Downloading - Downloading input data
2021-10-27 09:35:11 Training - Downloading the training image..................
2021-10-27 09:38:26 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-10-27 09:38:27,390 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2021-10-27 09:38:27,413 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2021-10-27 09:38:27,422 sagemaker_pytorch_container.training INFO     Invoking user training script.
2021-10-27 09:38:27,967 sagemaker-training-toolkit INFO     Installing dependencies fro

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


  Created wheel for nltk: filename=nltk-3.2.5-py3-none-any.whl size=1392142 sha256=3303cc85701ea1a6d407fe404570c72b5b0c536dcbf7a1e82cbb8b4ecf22b5b2
  Stored in directory: /root/.cache/pip/wheels/f2/7f/71/cb36468789a03b5e2908281c8e1ce093e6860258b6b61677d8
  Created wheel for sagemaker: filename=sagemaker-2.49.1-py2.py3-none-any.whl size=591916 sha256=1c4a3c7c6f517220af8cfc63998ba96ecd93684885679c63a33dbdcc095169c8
  Stored in directory: /root/.cache/pip/wheels/4c/af/ea/8ff5943a87155df5b184e54474fbf2b59b75e5c172854643c6
  Created wheel for aiobotocore: filename=aiobotocore-1.4.2-py3-none-any.whl size=49910 sha256=b6ec1b3d22a9870b36f409dd822965d83c775888a9599e84a1610b0e6ff5b38c
  Stored in directory: /root/.cache/pip/wheels/28/61/17/b5ccee30498ab9e21ed6ed0f65d71f79cd252017bc95b6fc00
  Created wheel for alembic: filename=alembic-1.4.1-py2.py3-none-any.whl size=158155 sha256=c9397d736b7ffe331977cfa97ddc909cd158df9972b77b0b4eff050b2972249a
  Stored in directory: /root/.cache/pip/wheels/e9/7b

[2021-10-27 09:40:21.277 algo-1:85 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2021-10-27 09:40:21.314 algo-1:85 INFO profiler_config_parser.py:102] User has disabled profiler.
[2021-10-27 09:40:21.315 algo-1:85 INFO json_config.py:91] Creating hook from json_config at /opt/ml/input/config/debughookconfig.json.
[2021-10-27 09:40:21.315 algo-1:85 INFO hook.py:201] tensorboard_dir has not been set for the hook. SMDebug will not be exporting tensorboard summaries.
[2021-10-27 09:40:21.316 algo-1:85 INFO hook.py:255] Saving to /opt/ml/output/tensors
[2021-10-27 09:40:21.316 algo-1:85 INFO state_store.py:77] The checkpoint config file /opt/ml/input/config/checkpointconfig.json does not exist.
[2021-10-27 09:40:21.426 algo-1:85 INFO hook.py:594] name:model.l0.embeddings.word_embeddings.weight count_params:7813632
[2021-10-27 09:40:21.427 algo-1:85 INFO hook.py:594] name:model.l0.embeddings.position_embeddings.weight count_params:131072
[2021-10-27 09:40:21.427 algo-1:85 INFO hook.

#015Validation sanity check: 0it [00:00, ?it/s]#015Validation sanity check:   0%|          | 0/2 [00:00<?, ?it/s]#015                                                              #015#015Training: 99it [00:00, ?it/s]#015Training:   0%|          | 0/7573 [00:00<?, ?it/s]#015Epoch 0:   0%|          | 0/7573 [00:00<?, ?it/s] #015Epoch 0:   0%|          | 30/7573 [00:01<08:09, 15.40it/s]#015Epoch 0:   0%|          | 30/7573 [00:01<08:09, 15.40it/s, loss=2.62, v_num=0, val_f1_epoch=0.0652, val_loss_epoch=0.668, train_f1=0.519]#015Epoch 0:   1%|          | 60/7573 [00:03<07:59, 15.68it/s, loss=2.62, v_num=0, val_f1_epoch=0.0652, val_loss_epoch=0.668, train_f1=0.519]#015Epoch 0:   1%|          | 60/7573 [00:03<07:59, 15.67it/s, loss=2.05, v_num=0, val_f1_epoch=0.0652, val_loss_epoch=0.668, train_f1=0.470]#015Epoch 0:   1%|          | 90/7573 [00:05<07:47, 16.00it/s, loss=2.05, v_num=0, val_f1_epoch=0.0652, val_loss_epoch=0.668, train_f1=0.470]#015Epoch 0:   1%|          | 90/7573 [00:05<07:47

...

#015Validating: 100%|██████████| 198/198 [00:11<00:00, 17.39it/s]#033[A#015Epoch 1: 100%|██████████| 3747/3747 [03:39<00:00, 17.04it/s, loss=1.2, v_num=0, val_f1_epoch=0.735, val_loss_epoch=0.293, train_f1=0.767, val_f1_step=0.816, val_loss_step=0.188] 
#015                                                             #033[A#015Epoch 1:   0%|          | 0/3747 [00:00<?, ?it/s, loss=1.2, v_num=0, val_f1_epoch=0.735, val_loss_epoch=0.293, train_f1=0.767, val_f1_step=0.816, val_loss_step=0.188]           #015Epoch 2:   0%|          | 0/3747 [00:00<?, ?it/s, loss=1.2, v_num=0, val_f1_epoch=0.735, val_loss_epoch=0.293, train_f1=0.767, val_f1_step=0.816, val_loss_step=0.188]#015Epoch 2:   1%|          | 30/3747 [00:02<04:09, 14.91it/s, loss=1.2, v_num=0, val_f1_epoch=0.735, val_loss_epoch=0.293, train_f1=0.767, val_f1_step=0.816, val_loss_step=0.188]#015Epoch 2:   1%|          | 30/3747 [00:02<04:09, 14.91it/s, loss=0.951, v_num=0, val_f1_epoch=0.735, val_loss_epoch=0.293, train_f1=0.779, val

oss=1.04, v_num=0, val_f1_epoch=0.735, val_loss_epoch=0.293, train_f1=0.758, val_f1_step=0.816, val_loss_step=0.188]#015Epoch 2:  36%|███▌      | 1350/3747 [01:19<02:20, 17.01it/s, loss=1.04, v_num=0, val_f1_epoch=0.735, val_loss_epoch=0.293, train_f1=0.758, val_f1_step=0.816, val_loss_step=0.188]#015Epoch 2:  36%|███▌      | 1350/3747 [01:19<02:20, 17.01it/s, loss=1.13, v_num=0, val_f1_epoch=0.735, val_loss_epoch=0.293, train_f1=0.767, val_f1_step=0.816, val_loss_step=0.188]#015Epoch 2:  37%|███▋      | 1380/3747 [01:21<02:19, 17.00it/s, loss=1.13, v_num=0, val_f1_epoch=0.735, val_loss_epoch=0.293, train_f1=0.767, val_f1_step=0.816, val_loss_step=0.188]#015Epoch 2:  37%|███▋      | 1380/3747 [01:21<02:19, 17.00it/s, loss=1.08, v_num=0, val_f1_epoch=0.735, val_loss_epoch=0.293, train_f1=0.710, val_f1_step=0.816, val_loss_step=0.188]#015Epoch 2:  38%|███▊      | 1410/3747 [01:22<02:17, 17.00it/s, loss=1.08, v_num=0, val_f1_epoch=0.735, val_loss_epoch=0.293, train_f1=0.710, val_f1_step=0

.28, v_num=0, val_f1_epoch=0.735, val_loss_epoch=0.293, train_f1=0.734, val_f1_step=0.816, val_loss_step=0.188] #015Epoch 2:  70%|███████   | 2640/3747 [02:35<01:05, 16.95it/s, loss=1.28, v_num=0, val_f1_epoch=0.735, val_loss_epoch=0.293, train_f1=0.734, val_f1_step=0.816, val_loss_step=0.188]#015Epoch 2:  70%|███████   | 2640/3747 [02:35<01:05, 16.95it/s, loss=0.995, v_num=0, val_f1_epoch=0.735, val_loss_epoch=0.293, train_f1=0.751, val_f1_step=0.816, val_loss_step=0.188]#015Epoch 2:  71%|███████▏  | 2670/3747 [02:37<01:03, 16.94it/s, loss=0.995, v_num=0, val_f1_epoch=0.735, val_loss_epoch=0.293, train_f1=0.751, val_f1_step=0.816, val_loss_step=0.188]#015Epoch 2:  71%|███████▏  | 2670/3747 [02:37<01:03, 16.94it/s, loss=1.32, v_num=0, val_f1_epoch=0.735, val_loss_epoch=0.293, train_f1=0.728, val_f1_step=0.816, val_loss_step=0.188] #015Epoch 2:  72%|███████▏  | 2700/3747 [02:39<01:01, 16.95it/s, loss=1.32, v_num=0, val_f1_epoch=0.735, val_loss_epoch=0.293, train_f1=0.728, val_f1_step=0.

#015Validating: 100%|██████████| 198/198 [00:11<00:00, 17.32it/s]#033[A#015Epoch 2: 100%|██████████| 3747/3747 [03:40<00:00, 16.98it/s, loss=1.11, v_num=0, val_f1_epoch=0.742, val_loss_epoch=0.288, train_f1=0.608, val_f1_step=0.816, val_loss_step=0.192]
#015                                                             #033[A#015Epoch 2: 100%|██████████| 3747/3747 [03:52<00:00, 16.12it/s, loss=1.11, v_num=0, val_f1_epoch=0.742, val_loss_epoch=0.288, train_f1=0.608, val_f1_step=0.816, val_loss_step=0.192]#015Epoch 2: 100%|██████████| 3747/3747 [05:22<00:00, 11.62it/s, loss=1.11, v_num=0, val_f1_epoch=0.742, val_loss_epoch=0.288, train_f1=0.608, val_f1_step=0.816, val_loss_step=0.192]
#015Epoch 2: 100%|██████████| 10238/10238 [1:33:45<00:00,  1.82it/s, loss=1.48, v_num=0, val_f1_epoch=0.571, val_loss_epoch=0.145, train_f1=0.494, val_f1_step=0.493, val_loss_step=0.0813]
2021-10-27 09:40:02.403976: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic li


2021-10-27 13:26:11 Uploading - Uploading generated training model
2021-10-27 13:30:12 Completed - Training job completed
Training seconds: 14115
Billable seconds: 14115


In [31]:
train_df = tot_df

In [30]:
tags_2d.intersection(tags_1d)

{'Capacities & Response->Number Of People Reached/Response Gaps',
 'Context->Economy',
 'Covid-19->Restriction Measures'}

In [21]:
assert (1==2)


AssertionError: 

## Code used for deploying and testing models:

In [ ]:
import mlflow
import torch

In [ ]:
model = mlflow.pytorch.load_model(
    's3://deep-mlflow-artifact/16/21a5ece6091b4ddf8b223e78159ce1c7/artifacts/pytorch_model_all',
    map_location=torch.device('cpu')   
)

In [ ]:
model.predict(test_data['excerpt'])

In [ ]:
from mlflow import sagemaker

In [ ]:
sagemaker.deploy(
    'testcpu-pytorch-trained-gpu',
    's3://deep-mlflow-artifact/16/21a5ece6091b4ddf8b223e78159ce1c7/artifacts/pytorch_model_all',
    execution_role_arn=SAGEMAKER_ROLE_ARN,
    image_url="961104659532.dkr.ecr.us-east-1.amazonaws.com/mlflow-pyfunc:latest",
    region_name="us-east-1",
    instance_type="ml.c4.xlarge",
    synchronous=False,
    archive=True,
)